In [1]:
import numpy as np
import matplotlib.pyplot as plt

import json

# Concepts
Since I am in a spherically symmetric spacetime, I can fix the angles to arbitrary values and forget them. So I only need to keep track of the radius. The 3D problem is effectively reduced to a 1D one!!! Radial differenciation is then reduced to simply the derivative along the array.

W.r.t. the eqs (3) in the reference, the variables are named as follows:
    $$ A \rightarrow \tilde{A} $$
    $$ B \rightarrow \tilde{B} $$
    $$ D_A \rightarrow \tilde{D}_A $$
    $$ D_B \rightarrow \tilde{D}_B $$

In [2]:
class Fields():
    
    def __init__(self, R = 1, N = 11, M = 1):
        # Domain is a sphere of radius R
        self.Rmax = R
        self.N    = N
        
        self.dR  = R / N
    
        self.r = np.array([(j - 0.5)*self.dR for j in range(1,N+1)], dtype = np.float64)

        self.A   = np.zeros(N)
        self.B   = np.zeros(N)
        self.DA  = np.zeros(N)
        self.DB  = np.zeros(N)
        self.KA  = np.zeros(N)
        self.KB  = np.zeros(N)
        self.Dal = np.zeros(N)
        self.al  = np.zeros(N)
        
        self.psi = 1 + 0.5 * M / self.r
        
    def IC_geodesicSlicing(self):
        """
        Geodesic Slicing gauge condition.
                A = B = 1
                DA = DB = 0
                KA = KB = 0
                alpha = 1 = costant
        """
        self.A  = np.ones(self.N)
        self.B  = np.ones(self.N)
        
        self.al = np.ones(self.N)
        
    def a_symmetryCond(self):
        self.A  [0] = self.A  [1]
        self.B  [0] = self.B  [1]
        self.DA [0] = -self.DA [1]
        self.DB [0] = -self.DB [1]
        self.KA [0] = self.KA [1]
        self.KB [0] = self.KB [1]
        
    def BC_Dirichlet(self):
        self.A[-1] = 1
        self.B[-1] = 1
        
        self.DA[-1] = 0
        self.DB[-1] = 0

In [3]:
def der_r(f, i, dR):
    if i == 0:
        # Forward
        return 0.5 * (- 3 * f[0] + 4 * f[1] - f[2]) / dR
    elif i == len(f)-1:
        # Backward
        return 0.5 * (3 * f[-1] - 4 * f[-2] + f[-3]) / dR
    else:
        # Central
        return 0.5 * (f[i+1] - f[i-1]) / dR
    
def sec_der_r(f, i, dR):
    if i == 0:
        # Forward
        return (2 * f[0] - 5 * f[1] + 4 * f[2] - f[3]) / dR**3
    elif i == len(f)-1:
        # Backward
        return (2 * f[-1] -5 * f[-2] + 4 * f[-3] - f[-4]) / dR**3
    else:
        # Central
        return (f[i+1] - 2 * f[i] + f[i-1]) / dR**2

In [4]:
def ev_A(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i] * field.A[i] * field.KA[i]

def ev_B(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    return - 2 * field.al[i] * field.B[i] * field.KB[i]

def ev_DA(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KA[i] * field.Dal[i]
    p2 = der_r(field.KA, i, field.dR)
    return - 2 * field.al[i] * (p1 + p2)

def ev_DB(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    p1 = field.KB[i] * field.Dal[i]
    p2 = der_r(field.KB, i, field.dR)
    return - 2 * field.al[i] * (p1 + p2)


def ev_KA(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    r = field.r[i]
    
    p1 = der_r(field.Dal + field.DB, i, field.dR)
    p2 = field.Dal[i]**2 + 0.5 * (field.Dal[i] * field.DA[i] + field.DB[i]**2 - field.DA[i] * field.DB[i])
    p3 = - field.A[i] * field.KA[i] * (field.KA[i] + 2*field.KB[i]) * field.psi[i]**4
    p4 = - (field.DA[i] - 2 * field.DB[i]) / r
    p5 = 4 * sec_der_r(np.log(field.psi), i, field.dR) + der_r(np.log(field.psi), i, field.dR) * (2 * field.DB[i] - 2 * field.DA[i] - 2 * field.Dal[i]  + 4 / r)
    return - field.al[i] * (p1 + p2 + p3 + p4 + p5) / (field.A[i] * field.psi[i]**4)
    
def ev_KB(field, i):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    r = field.r[i]
    
    p1 = der_r(field.DB, i, field.dR)
    p2 = field.Dal[i] * field.DB[i] + field.DB[i]**2 - 0.5 * field.DA[i] * field.DB[i]
    p3 = - (field.DA[i] - 2 * field.Dal[i] - 4 * field.DB[i]) / r
    p4 = - 2 * (field.A[i] - field.B[i]) / (field.B[i] * r**2)
    p5 = 4 * sec_der_r(np.log(field.psi), i, field.dR) + der_r(field.psi, i, field.dR) * (8*der_r(field.psi, i, field.dR) + 4*field.Dal[i] + 6*field.DB[i] -2*field.DA[i] + 12 / r)
    p6 = field.al[i] * field.KB[i] * (field.KA[i] + 2*field.KB[i])
    return - 0.5 * field.al[i] * (p1 + p2 + p3 + p4 + p5) / field.A[i] + p6
    

In [25]:
import h5py

In [77]:
dt = 0.01
t = 0
N = 100

h5f = h5py.File('results.h5', 'w')
sss = f'Each dataset contains the following variables:\n \
\tA\n\
\tB\n\
\tD_A\n\
\tD_B\n\
\tK_A\n\
\tK_B\n\
\tD_alpha\n\
\talpha\n\
\nThe name of the dataset is the index of the time step (e.g. "i" is the first computation, at t = i*dt)\n\
Parameters:\n\
\tN = {N} (number of points in the radial direction)\n\
\tdt = {dt} (timestep used)\
'
h5f.attrs['Info']  = sss

fields = Fields(R = 1, N = N)
fields.IC_geodesicSlicing()

results = np.zeros((8, N))

for j in range(10):
    t += dt
    
    fields.a_symmetryCond()
    fields.BC_Dirichlet()
    for i in range(N):

        results[0,i] = fields.A [i] + dt * ev_A (fields, i) #  A_new  
        results[1,i] = fields.B [i] + dt * ev_B (fields, i) #  B_new 
        results[2,i] = fields.DA[i] + dt * ev_DA(fields, i) #  DA_new
        results[3,i] = fields.DB[i] + dt * ev_DB(fields, i) #  DB_new 
        results[4,i] = fields.KA[i] + dt * ev_KA(fields, i) #  KA_new 
        results[5,i] = fields.KB[i] + dt * ev_KB(fields, i) #  KB_new 
        results[6,i] = fields.al[i]                         #  Dal_new
        results[7,i] = fields.Dal[i]                        #  al_new
    
    h5f.create_dataset(f'{j}', data=results, compression = 9)
    
    fields.A  = np.copy(results[0,:])
    fields.B  = np.copy(results[1,:])
    fields.DA = np.copy(results[2,:])
    fields.DB = np.copy(results[3,:])
    fields.KA = np.copy(results[4,:])
    fields.KB = np.copy(results[5,:])
    
h5f.close() 
    
    

In [61]:
fields.A

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [62]:
aaa = h5py.File('results.h5','r')

In [63]:
list(aaa.values())

[<HDF5 dataset "t0": shape (8, 10), type "<f8">,
 <HDF5 dataset "t1": shape (8, 10), type "<f8">]

In [67]:
list(aaa.attrs)

['Info']

In [76]:
aaa.attrs['Info']

'Each dataset contains the following variables:\n \tA\n\tB\n\tD_A\n\tD_B\n\tK_A\n\tK_B\n\tD_alpha\n\talpha\n\nThe name of the dataset is the index of the time step (e.g. "i" is the first computation, at t = i*dt)\nParameters:\n\tN = 10 (number of points in the radial direction)\n\tdt = 0.01 (timestep used)'

In [72]:
list(aaa['t0'])

[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]),
 array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]),
 array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4.]),
 array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]),
 array([6., 6., 6., 6., 6., 6., 6., 6., 6., 6.]),
 array([7., 7., 7., 7., 7., 7., 7., 7., 7., 7.]),
 array([8., 8., 8., 8., 8., 8., 8., 8., 8., 8.])]